In [ ]:
import pandas as pd
import numpy as np
import re
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer

print("✅ Kütüphaneler yüklendi!")

In [ ]:
# Veriyi yükle
df = pd.read_csv('../data/raw/music_reviews_20251224_031752.csv')
print(f"📊 Toplam {len(df)} inceleme yüklendi")
print(f"\nKolonlar: {list(df.columns)}")
df.head()

In [ ]:
# Text cleaning function
def clean_text(text):
    if pd.isna(text):
        return ""
    text = str(text).lower()
    text = re.sub(r'http\S+|www\S+', '', text)  # URLs
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Special chars
    text = re.sub(r'\s+', ' ', text).strip()  # Extra spaces
    return text

# İncelemeleri temizle
text_column = 'content' if 'content' in df.columns else 'review_text'
df['cleaned_text'] = df[text_column].apply(clean_text)

# Çok kısa incelemeleri çıkar
df_clean = df[df['cleaned_text'].str.len() >= 20].copy()
df_clean = df_clean.drop_duplicates(subset='cleaned_text').reset_index(drop=True)

print(f"🧹 {len(df_clean)} temiz inceleme hazır")
df_clean[['cleaned_text', 'score']].head()

In [ ]:
# BERTopic model oluştur
print("🤖 BERTopic modeli oluşturuluyor...")

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
vectorizer_model = CountVectorizer(stop_words='english', min_df=2, ngram_range=(1, 2))

topic_model = BERTopic(
    embedding_model=embedding_model,
    vectorizer_model=vectorizer_model,
    min_topic_size=20,
    nr_topics='auto',
    verbose=True
)

print("✅ Model hazır!")

In [ ]:
# Topic modeling yap
print("📊 Topic modeling başlıyor... (Bu birkaç dakika sürebilir)")

documents = df_clean['cleaned_text'].tolist()
topics, probs = topic_model.fit_transform(documents)

print(f"\n✅ {len(set(topics))-1} topic bulundu! (-1 outliers hariç)")

In [ ]:
# Topic bilgilerini gör
topic_info = topic_model.get_topic_info()
print("📊 Topic Dağılımı:")
topic_info.head(15)

In [ ]:
# Topic'leri anlamlı kategorilere çevir
def categorize_topic(topic_id, topic_words):
    """Topic kelimelerine göre kategori ata"""
    if topic_id == -1:
        return "Outliers"
    
    words = ' '.join(topic_words)
    
    # Bug/Error keywords
    if any(word in words for word in ['crash', 'bug', 'error', 'broken', 'fix', 'problem', 'issue']):
        return "🐛 Bugs & Errors"
    
    # Login/Account keywords
    if any(word in words for word in ['login', 'account', 'password', 'sign', 'authentication']):
        return "🔐 Login & Account Issues"
    
    # UI/UX keywords
    if any(word in words for word in ['interface', 'design', 'ui', 'ux', 'layout', 'menu']):
        return "🎨 UI/UX Issues"
    
    # Feature Request keywords
    if any(word in words for word in ['feature', 'add', 'want', 'need', 'wish', 'request']):
        return "💡 Feature Requests"
    
    # Music/Audio keywords
    if any(word in words for word in ['music', 'song', 'audio', 'sound', 'quality', 'playlist']):
        return "🎵 Music & Audio Quality"
    
    # Subscription/Billing keywords
    if any(word in words for word in ['subscription', 'premium', 'pay', 'billing', 'price', 'cost']):
        return "💳 Subscription & Billing"
    
    # Performance keywords
    if any(word in words for word in ['slow', 'lag', 'performance', 'speed', 'loading']):
        return "⚡ Performance Issues"
    
    # Positive feedback
    if any(word in words for word in ['love', 'great', 'amazing', 'best', 'perfect', 'excellent']):
        return "⭐ Positive Feedback"
    
    return f"📁 Topic {topic_id}"

# Her topic için kategori ata
topic_categories = {}
for idx, row in topic_info.iterrows():
    topic_id = row['Topic']
    # Topic kelimelerini al (ilk 10 kelime)
    if topic_id != -1:
        topic_words = [word for word, _ in topic_model.get_topic(topic_id)[:10]]
    else:
        topic_words = []
    category = categorize_topic(topic_id, topic_words)
    topic_categories[topic_id] = category

print("✅ Topic kategorileri oluşturuldu!")
for tid, cat in list(topic_categories.items())[:10]:
    print(f"Topic {tid}: {cat}")

In [ ]:
# Sonuçları dataframe'e ekle
df_clean['topic'] = topics
df_clean['topic_category'] = df_clean['topic'].map(topic_categories)
df_clean['topic_probability'] = [p.max() if len(p) > 0 else 0 for p in probs]

print("\n📊 Kategori Dağılımı:")
print(df_clean['topic_category'].value_counts())

df_clean[['cleaned_text', 'score', 'topic_category', 'topic_probability']].head(10)

In [ ]:
# Sonuçları kaydet
import os

os.makedirs('../data/processed', exist_ok=True)
output_path = '../data/processed/reviews_with_topics.csv'
df_clean.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"✅ Sonuçlar kaydedildi: {output_path}")
print(f"\n📈 Özet:")
print(f"  - Toplam işlenen inceleme: {len(df_clean):,}")
print(f"  - Bulunan kategori sayısı: {df_clean['topic_category'].nunique()}")
print(f"  - En büyük kategori: {df_clean['topic_category'].value_counts().index[0]}")

In [ ]:
# Her kategoriden örnek incelemelere bak
print("\n🔍 Her kategoriden örnek incelemeler:\n")
for category in df_clean['topic_category'].value_counts().head(5).index:
    print(f"\n{'='*60}")
    print(f"{category}")
    print(f"{'='*60}")
    samples = df_clean[df_clean['topic_category'] == category].head(3)
    for idx, row in samples.iterrows():
        print(f"⭐ {row['score']}/5: {row['cleaned_text'][:150]}...")
        print()